## Object Detection using Cascade Classifiers

In [1]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Load image
img = cv2.imread('justin_face.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Load existing cascade classifier for face and eyes
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

In [3]:
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.8, 6)

# For each face detected draw a detection rectangle in blue (255,0,0)
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    
    # Detect eyes (within face)
    eyes = eye_cascade.detectMultiScale(roi_gray)
    
    # For each set of eyes detected draw a detection rectangle in green (0,255,0)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

In [4]:
# Show image with detection rectangles drawn        
cv2.imshow('img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Object Detection using YOLO

In [5]:
# Import required libraries
import numpy as np
import cv2

In [6]:
# Load the COCO class labels our YOLO model was trained on. (80 labels)
LABELS = open("yolov3.txt").read().strip().split("\n")

# Initialize list of colors to represent class labels, use bright colors so its easier to read
COLORS = np.random.randint(150, 255, size=(len(LABELS), 3), dtype="uint8")

In [7]:
# Load our YOLO object detector trained on COCO dataset (80 classes)
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')

In [8]:
# Load our input image and grab dimensions
image = cv2.imread("object_detection.jpg")
(H, W) = image.shape[:2]
 
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
 
# Construct a blob from the input image and perform forward pass of YOLO to get bounding boxes and probabilities
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layerOutputs = net.forward(ln)

In [9]:
# Initialize our lists of detected bounding boxes, confidences, and class IDs, respectively
boxes = []
confidences = []
classIDs = []

In [10]:
# Loop over each of the layer outputs
for output in layerOutputs:
    # Loop over each of the detections
    for detection in output:
        # Extract the class ID and confidence of the current object detection
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        # Filter out weak predictions by ensuring detected probability is greater some threshold
        if confidence > 0.5:
            # Scale bounding box coordinates back relative to size of image
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY, width, height) = box.astype("int")

            # Use the center (x, y)-coordinates to derive top left corner of the bounding box
            x = int(centerX - (width / 2))
            y = int(centerY - (height / 2))

            # Update list of bounding box coordinates, confidences, and class IDs
            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)

In [11]:
# Apply non-maximum suppression to suppress weak overlapping bounding boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [12]:
# Loop over the indexes and draw bounding boxes
for i in idxs.flatten():
    # Extract bounding box coordinates
    (x, y) = (boxes[i][0], boxes[i][1])
    (w, h) = (boxes[i][2], boxes[i][3])
 
    # Draw a bounding box using opencv rectangle and label the image
    color = [int(c) for c in COLORS[classIDs[i]]]
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [13]:
# show the output image
cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()